## importing data

In [1]:
import scipy.io as sio
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%pprint

Pretty printing has been turned OFF


In [3]:
start_dir = "../../"+os.path.relpath(os.path.expanduser("~"))+"/Volumes/project/Hanson"

In [4]:
floc = "/External_MRI_Projects/NKI_Rockland/Neuroimaging/BIDS/derivatives/DWI_mrtrix_preprocessed"

In [5]:
[ x for x in os.listdir(start_dir+floc) if x.endswith("mat") ]

['Fingerprint_test_connectometry.qa.db.fib.gz.vec0.mat', 'FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec0.mat', 'FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec1.mat']

In [6]:
test_fn = '/Fingerprint_test_connectometry.qa.db.fib.gz.vec0.mat'
real_fn1 = '/FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec0.mat'
real_fn2 = '/FingerPrintTest.N677.connectometry.qa.db.fib.gz.vec1.mat'

In [7]:
# matfile1 = sio.loadmat(start_dir+floc+real_fn1)
# matfile2 = sio.loadmat(start_dir+floc+real_fn2)

In [8]:
# joblib is the same as pickle but more efficient for stuff w numpy arrays
from joblib import dump, load

In [9]:
# dump(matfile1, '../files/mat1.joblib',compress=3) 
# dump(matfile2, '../files/mat2.joblib',compress=3) 

In [10]:
matfile1 = load('../files/mat1.joblib') 
matfile2 = load('../files/mat2.joblib') 

In [11]:
list(matfile1.keys())[:20]

['subject_names', 'subject0', 'subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15', 'subject16', 'subject17', 'subject18']

In [12]:
list(matfile1.keys())[-20:]

['subject384', 'subject385', 'subject386', 'subject387', 'subject388', 'subject389', 'subject390', 'subject391', 'subject392', 'subject393', 'subject394', 'subject395', 'subject396', 'subject397', 'subject398', 'subject399', 'dimension', 'voxel_location', 'mni_location', 'fiber_direction']

In [13]:
# why is this a nested array???
matfile1["subject_names"][0]

array([115, 117,  98, ..., 102,  10,   0], dtype=uint8)

In [14]:
# phenotypic data
pheno_df = pd.read_csv("../files/NKI_IRI_DWI_combined.csv")

In [15]:
pheno_df.head()

,Identifiers_alt,age,sex,ipri_29,ipri_30,ipri_31,ipri_32
0,sub-A00023510_ses-BAS1,23,0,17,21,19,17
1,sub-A00027439_ses-BAS1,15,0,17,11,14,8
2,sub-A00027443_ses-BAS1,15,0,11,9,16,5
3,sub-A00027544_ses-BAS1,22,0,22,19,24,16
4,sub-A00028150_ses-BAS1,44,1,12,18,19,17


In [16]:
pheno_df.describe()

,age,sex,ipri_29,ipri_30,ipri_31,ipri_32
count,677.000000,677.000000,677.000000,677.000000,677.000000,677.000000
mean,32.324963,0.604136,15.877400,14.422452,20.404727,11.707533
std,13.110167,0.489397,4.216554,6.009014,5.072420,5.204699
min,13.000000,0.000000,3.000000,0.000000,0.000000,1.000000
25%,21.000000,0.000000,13.000000,10.000000,17.000000,8.000000
50%,30.000000,1.000000,16.000000,15.000000,21.000000,12.000000
75%,45.000000,1.000000,19.000000,19.000000,24.000000,15.000000
max,54.000000,1.000000,24.000000,28.000000,28.000000,31.000000


## prepping the data

In [17]:
# just the subjects
subj1 = [x for x in list(matfile1.keys()) if "sub" in x and "_" not in x]
subj2 = [x for x in list(matfile2.keys()) if "sub" in x and "_" not in x]

In [18]:
len(subj1)

400

In [19]:
len(subj2)

277

In [20]:
X = []
for sub in subj1:
    X.append(matfile1[sub][0])
for sub in subj2:
    X.append(matfile2[sub][0])

In [21]:
X = np.array(X)
X

array([[0.13154687, 0.14433745, 0.1312975 , ..., 0.03943831, 0.02088308,
        0.05409636],
       [0.8178366 , 0.74562216, 0.72506034, ..., 0.34529442, 0.41764247,
        0.56341   ],
       [0.50131667, 0.47282678, 0.5085435 , ..., 0.1921919 , 0.2769494 ,
        0.2239489 ],
       ...,
       [2.193593  , 1.8504024 , 2.1619565 , ..., 0.5081969 , 0.4759148 ,
        0.6066462 ],
       [0.2378414 , 0.17248623, 0.23666348, ..., 0.8264794 , 0.7652172 ,
        0.7216453 ],
       [0.32788658, 0.35372517, 0.36588156, ..., 0.14133348, 0.10620189,
        0.06241945]], dtype=float32)

In [22]:
y = np.array(pheno_df.age)

In [23]:
# already saved
# dump(X, '../files/fingerprintX.joblib',compress=3) 
# dump(y, '../files/fingerprintY.joblib',compress=3) 

## making the data smaller

In [24]:
len(X[0])

319861

In [25]:
N_COMPONENTS = len(X)

In [26]:
# PCA is preferable but depending on the shape of the data i might use truncatedSVD
from sklearn.decomposition import PCA, TruncatedSVD

In [27]:
pca = PCA(n_components=N_COMPONENTS, random_state=0)
X_pca = pca.fit_transform(X)

In [28]:
# sweet
len(X_pca[0])

677

In [29]:
#dump(X_pca, '../files/fingerprintX_pca.joblib',compress=3) 

## fitting the model(s)

In [30]:
len(X_pca)

677

In [55]:
y = np.array(pheno_df.age)
len(y)

677

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size = 1/3, random_state = 0)

In [69]:
from sklearn.metrics import r2_score

In [83]:
#all_labels = model.predict(X_pca)

In [84]:
#pheno_df["pred_age"]=all_labels

In [85]:
#pheno_df.head()

In [86]:
#sns.relplot(x="age",y="pred_age", data=pheno_df)

In [87]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [88]:
all_gs = load('../files/fingerprint_all_gs.joblib')

In [89]:
#svr_gs.cv_results_
all_gs.best_params_

{'model': ElasticNet(l1_ratio=0.01, random_state=0), 'model__alpha': 1.0, 'model__l1_ratio': 0.01, 'model__random_state': 0, 'model__selection': 'cyclic'}

In [90]:
dir(all_gs)

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_is_fitted', '_check_n_features', '_check_refit_for_multimetric', '_estimator_type', '_format_results', '_get_param_names', '_get_tags', '_more_tags', '_pairwise', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_run_search', '_validate_data', 'best_estimator_', 'best_index_', 'best_params_', 'best_score_', 'classes_', 'cv', 'cv_results_', 'decision_function', 'error_score', 'estimator', 'fit', 'get_params', 'inverse_transform', 'multimetric_', 'n_features_in_', 'n_jobs', 'n_splits_', 'param_grid', 'pre_dispatch', 'predict', 'predict_log_pro

In [91]:
all_gs.scoring

make_scorer(mean_squared_error, greater_is_better=False)

In [92]:
means = all_gs.cv_results_["mean_test_score"]
stds = all_gs.cv_results_["std_test_score"]
results = sorted(list(zip(means, stds, all_gs.cv_results_["params"])), 
                 key = lambda x: x[0], reverse=True)

In [93]:
for mean, std, params in results[:10]:
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

-44.969 (+/-15.279) for {'model': ElasticNet(l1_ratio=0.01, random_state=0), 'model__alpha': 1.0, 'model__l1_ratio': 0.01, 'model__random_state': 0, 'model__selection': 'cyclic'}

-45.191 (+/-15.276) for {'model': ElasticNet(l1_ratio=0.01, random_state=0), 'model__alpha': 1.0, 'model__l1_ratio': 0.01, 'model__random_state': 0, 'model__selection': 'random'}

-47.076 (+/-11.846) for {'model': ElasticNet(l1_ratio=0.01, random_state=0), 'model__alpha': 1.0, 'model__l1_ratio': 0.1, 'model__random_state': 0, 'model__selection': 'cyclic'}

-47.161 (+/-11.908) for {'model': ElasticNet(l1_ratio=0.01, random_state=0), 'model__alpha': 1.0, 'model__l1_ratio': 0.1, 'model__random_state': 0, 'model__selection': 'random'}

-47.407 (+/-16.307) for {'model': ElasticNet(l1_ratio=0.01, random_state=0), 'model__alpha': 1.0, 'model__l1_ratio': 0.001, 'model__random_state': 0, 'model__selection': 'cyclic'}

-47.459 (+/-14.403) for {'model': ElasticNet(l1_ratio=0.01, random_state=0), 'model__alpha': 0.01, 'm

In [119]:
results[9][2]

{'model': Ridge(), 'model__alpha': 1.0, 'model__random_state': 0, 'model__solver': 'saga'}

In [120]:
model1 = ElasticNet(l1_ratio=0.01, random_state=0, alpha=1.0, selection='cyclic')
model2 = ElasticNet(l1_ratio=0.01, random_state=0, alpha=1.0, selection='random')
model3 = ElasticNet(l1_ratio=0.1, random_state=0, alpha=1.0, selection='cyclic')
model4 = ElasticNet(l1_ratio=0.1, random_state=0, alpha=1.0, selection='random')
model5 = ElasticNet(l1_ratio=0.001, random_state=0, alpha=1.0, selection='cyclic')
model6 = ElasticNet(l1_ratio=0.1, random_state=0, alpha=0.01, selection='cyclic')
model7 = ElasticNet(l1_ratio=0.001, random_state=0, alpha=1.0, selection='random')
model8 = Ridge(alpha=1.0, random_state=0,solver='lsqr')
model9 = Ridge(alpha=0.0001, random_state=0,solver='saga')
model10 = Ridge(alpha=1.0, random_state=0,solver='saga')

In [122]:
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)
model4.fit(X_train,y_train)
model5.fit(X_train,y_train)
model6.fit(X_train,y_train)
model7.fit(X_train,y_train)
model8.fit(X_train,y_train)
model9.fit(X_train,y_train)
model10.fit(X_train,y_train)

/Users/evb32/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.131092071533203, tolerance: 7.779956817626953
  model = cd_fast.enet_coordinate_descent(


Ridge(random_state=0, solver='saga')

In [125]:
model1.get_params()

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.01, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}

In [126]:
for x in [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10]:
    y_true, y_pred = y_test, x.predict(X_test)
    print(x.get_params())
    print(r2_score(y_true, y_pred))
    print()

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.01, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
0.7915343965610114

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.01, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'random', 'tol': 0.0001, 'warm_start': False}
0.7902410956058957

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.1, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
0.7382594450329123

{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.1, 'max_iter': 1000, 'normalize': False, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'random', 'tol': 0.0001, 'warm_start': False}
0.7369658805949687

{'alpha': 

In [59]:
y_true, y_pred = y_test, all_gs.predict(X_test)

In [71]:
r2_score(y_true, y_pred)

0.7915343965610114